In [1]:
import sys
print(sys.path)
sys.path.append("C:/Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines/") #need to do this cuz otherwise ieeg isn't added to path...

from ieeg.navigate import channel_outlier_marker, trial_ieeg, crop_empty_data, \
    outliers_to_nan
from ieeg.io import raw_from_layout, get_data
from ieeg.timefreq.utils import crop_pad
from ieeg.timefreq import gamma
from ieeg.calc.scaling import rescale
import mne
import os
import numpy as np
import pandas as pd
from ieeg.calc.reshape import make_data_same
from ieeg.calc.stats import time_perm_cluster
from ieeg.calc.mat import LabeledArray

from utils import *
import matplotlib.pyplot as plt

from pandas import read_csv
import scipy.stats as stats
import joblib

from scipy.ndimage import label

import json
import pickle

# rsa toolbox imports
from rsatoolbox.io.mne import read_epochs
from rsatoolbox.data.ops import merge_datasets
from rsatoolbox.rdm import calc_rdm_movie
from rsatoolbox.rdm.calc import _parse_input
from rsatoolbox.util.build_rdm import _build_rdms
from rsatoolbox.rdm import compare
from rsatoolbox.vis import show_rdm
from rsatoolbox.vis.timecourse import plot_timecourse

from os.path import join, expanduser, basename
import glob, json
import numpy, tqdm, mne, pandas
import rsatoolbox
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot

from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from ieeg.decoding.decoders import PcaLdaClassification
from ieeg.calc.oversample import MinimumNaNSplit
from ieeg.calc.fast import mixup
from jim_decoding_functions import *


['C:\\Users\\jz421\\Desktop\\GlobalLocal\\IEEG_Pipelines', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\python311.zip', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\DLLs', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg', '', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32\\lib', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\Pythonwin', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\Pythonwin']




Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
subjects = ['D0057','D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094', 'D0100', 'D0102', 'D0103']
# load in subjects electrodes to rois dict. If it doesn't already exist, make it and then load it.
filename = 'subjects_electrodestoROIs_dict.json'
subjects_electrodestoROIs_dict = make_or_load_subjects_electrodes_to_rois_dict(filename, subjects)

Attempting to load the subjects' electrodes-to-ROIs dictionary...
Loaded data from subjects_electrodestoROIs_dict.json
Dictionary loaded successfully. Ready to proceed!


In [3]:
stimulus_experiment_conditions = {
    "Stimulus/i25.0/s25.0": {
        "BIDS_events": "Stimulus/i25.0/s25.0",
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Stimulus/i25.0/s75.0": {
        "BIDS_events": "Stimulus/i25.0/s75.0",
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Stimulus/i75.0/s25.0": {
        "BIDS_events": "Stimulus/i75.0/s25.0",
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Stimulus/i75.0/s75.0": {
        "BIDS_events": "Stimulus/i75.0/s75.0",
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Stimulus/i25.0/r25.0": {
        "BIDS_events": "Stimulus/i25.0/r25.0",
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Stimulus/i25.0/r75.0": {
        "BIDS_events": "Stimulus/i25.0/r75.0",
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "75%"
    },
    "Stimulus/i75.0/r25.0": {
        "BIDS_events": "Stimulus/i75.0/r25.0",
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Stimulus/i75.0/r75.0": {
        "BIDS_events": "Stimulus/i75.0/r75.0",
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "75%"
    },
    "Stimulus/c25.0/s25.0": {
        "BIDS_events": "Stimulus/c25.0/s25.0",
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Stimulus/c25.0/s75.0": {
        "BIDS_events": "Stimulus/c25.0/s75.0",
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Stimulus/c75.0/s25.0": {
        "BIDS_events": "Stimulus/c75.0/s25.0",
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Stimulus/c75.0/s75.0": {
        "BIDS_events": "Stimulus/c75.0/s75.0",
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Stimulus/c25.0/r25.0": {
        "BIDS_events": "Stimulus/c25.0/r25.0",
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Stimulus/c25.0/r75.0": {
        "BIDS_events": "Stimulus/c25.0/r75.0",
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "75%"
    },
    "Stimulus/c75.0/r25.0": {
        "BIDS_events": "Stimulus/c75.0/r25.0",
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Stimulus/c75.0/r75.0": {
        "BIDS_events": "Stimulus/c75.0/r75.0",
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "75%"
    }
}

# congruency_conditions = {
#     "Stimulus_c": {
#         "BIDS_events": ["Stimulus/c25/s25", "Stimulus/c25/s75", "Stimulus/c75/s25", "Stimulus/c75/s75", "Stimulus/c25/r25", "Stimulus/c25/r75", "Stimulus/c75/r25", "Stimulus/c75/r75"],
#         "congruency": "c"
#     }
# }

stimulus_conditions = {
    "Stimulus/BigLetters/SmallLetterh/Taskg": {
        "BIDS_events": "Stimulus/BigLetters/SmallLetterh/Taskg",
        "bigLetter": "s",
        "smallLetter": "h",
        "task": "g"
    },
    "Stimulus/BigLetters/SmallLetterh/Taskl": {
        "BIDS_events": "Stimulus/BigLetters/SmallLetterh/Taskl",
        "bigLetter": "s",
        "smallLetter": "h",
        "task": "l"
    },
    "Stimulus/BigLetters/SmallLetters/Taskg": {
        "BIDS_events": "Stimulus/BigLetters/SmallLetters/Taskg",
        "bigLetter": "s",
        "smallLetter": "s",
        "task": "g"
    },
    "Stimulus/BigLetters/SmallLetters/Taskl": {
        "BIDS_events": "Stimulus/BigLetters/SmallLetters/Taskl",
        "bigLetter": "s",
        "smallLetter": "s",
        "task": "l"
    },
    "Stimulus/BigLetterh/SmallLetterh/Taskg": {
        "BIDS_events": "Stimulus/BigLetterh/SmallLetterh/Taskg",
        "bigLetter": "h",
        "smallLetter": "h",
        "task": "g"
    },
    "Stimulus/BigLetterh/SmallLetterh/Taskl": {
        "BIDS_events": "Stimulus/BigLetterh/SmallLetterh/Taskl",
        "bigLetter": "h",
        "smallLetter": "h",
        "task": "l"
    },
    "Stimulus/BigLetterh/SmallLetters/Taskg": {
        "BIDS_events": "Stimulus/BigLetterh/SmallLetters/Taskg",
        "bigLetter": "h",
        "smallLetter": "s",
        "task": "g"
    },
    "Stimulus/BigLetterh/SmallLetters/Taskl": {
        "BIDS_events": "Stimulus/BigLetterh/SmallLetters/Taskl",
        "bigLetter": "h",
        "smallLetter": "s",
        "task": "l"
    }
}

response_experiment_conditions = {
    "Response/i25.0/s25.0": {
        "BIDS_events": "Response/i25.0/s25.0",
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Response/i25.0/s75.0": {
        "BIDS_events": "Response/i25.0/s75.0",
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Response/i75.0/s25.0": {
        "BIDS_events": "Response/i75.0/s25.0",
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Response/i75.0/s75.0": {
        "BIDS_events": "Response/i75.0/s75.0",
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Response/i25.0/r25.0": {
        "BIDS_events": "Response/i25.0/r25.0",
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Response/i25.0/r75.0": {
        "BIDS_events": "Response/i25.0/r75.0",
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "75%"
    },
    "Response/i75.0/r25.0": {
        "BIDS_events": "Response/i75.0/r25.0",
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Response/i75.0/r75.0": {
        "BIDS_events": "Response/i75.0/r75.0",
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "75%"
    },
    "Response/c25.0/s25.0": {
        "BIDS_events": "Response/c25.0/s25.0",
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Response/c25.0/s75.0": {
        "BIDS_events": "Response/c25.0/s75.0",
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Response/c75.0/s25.0": {
        "BIDS_events": "Response/c75.0/s25.0",
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Response/c75.0/s75.0": {
        "BIDS_events": "Response/c75.0/s75.0",
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Response/c25.0/r25.0": {
        "BIDS_events": "Response/c25.0/r25.0",
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Response/c25.0/r75.0": {
        "BIDS_events": "Response/c25.0/r75.0",
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "75%"
    },
    "Response/c75.0/r25.0": {
        "BIDS_events": "Response/c75.0/r25.0",
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Response/c75.0/r75.0": {
        "BIDS_events": "Response/c75.0/r75.0",
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "75%"
    }
}

response_conditions = {
    "Response/BigLetters/SmallLetterh/Taskg": {
        "BIDS_events": "Response/BigLetters/SmallLetterh/Taskg",
        "bigLetter": "s",
        "smallLetter": "h",
        "task": "g"
    },
    "Response/BigLetters/SmallLetterh/Taskl": {
        "BIDS_events": "Response/BigLetters/SmallLetterh/Taskl",
        "bigLetter": "s",
        "smallLetter": "h",
        "task": "l"
    },
    "Response/BigLetters/SmallLetters/Taskg": {
        "BIDS_events": "Response/BigLetters/SmallLetters/Taskg",
        "bigLetter": "s",
        "smallLetter": "s",
        "task": "g"
    },
    "Response/BigLetters/SmallLetters/Taskl": {
        "BIDS_events": "Response/BigLetters/SmallLetters/Taskl",
        "bigLetter": "s",
        "smallLetter": "s",
        "task": "l"
    },
    "Response/BigLetterh/SmallLetterh/Taskg": {
        "BIDS_events": "Response/BigLetterh/SmallLetterh/Taskg",
        "bigLetter": "h",
        "smallLetter": "h",
        "task": "g"
    },
    "Response/BigLetterh/SmallLetterh/Taskl": {
        "BIDS_events": "Response/BigLetterh/SmallLetterh/Taskl",
        "bigLetter": "h",
        "smallLetter": "h",
        "task": "l"
    },
    "Response/BigLetterh/SmallLetters/Taskg": {
        "BIDS_events": "Response/BigLetterh/SmallLetters/Taskg",
        "bigLetter": "h",
        "smallLetter": "s",
        "task": "g"
    },
    "Response/BigLetterh/SmallLetters/Taskl": {
        "BIDS_events": "Response/BigLetterh/SmallLetters/Taskl",
        "bigLetter": "h",
        "smallLetter": "s",
        "task": "l"
    }
}

In [4]:
task='GlobalLocal'

conditions = stimulus_conditions # toggle

if conditions == stimulus_conditions or stimulus_experiment_conditions:
    epochs_root_file = "Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8"
elif conditions == response_conditions or response_experiment_conditions:
    epochs_root_file = "Response_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8"
else:
    raise ValueError("Unknown condition type.")

condition_names = list(conditions.keys()) # get the condition names as a list

# This breaks if just_HG_ev1_rescaled is set to False currently. Fix this! 8/11
subjects_mne_objects = create_subjects_mne_objects_dict(subjects=subjects, epochs_root_file=epochs_root_file, conditions=conditions, task="GlobalLocal", just_HG_ev1_rescaled=True, acc_trials_only=True)

Loading data for subject: D0057
Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\D0057\D0057_Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_HG_ev1_rescaled-epo.fif ...
    Found the data of interest:
        t =   -1000.00 ...    1500.00 ms
        0 CTF compensation matrices available
Not setting metadata
449 matching events found
No baseline correction applied
0 projection items activated
  Loading condition: Stimulus/BigLetters/SmallLetterh/Taskg with parameters: {'BIDS_events': 'Stimulus/BigLetters/SmallLetterh/Taskg', 'bigLetter': 's', 'smallLetter': 'h', 'task': 'g'}
  Loading condition: Stimulus/BigLetters/SmallLetterh/Taskl with parameters: {'BIDS_events': 'Stimulus/BigLetters/SmallLetterh/Taskl', 'bigLetter': 's', 'smallLetter': 'h', 'task': 'l'}
  Loading condition: Stimulus/BigLetters/SmallLetters/Taskg with parameters: {'BIDS_events': 'Stimulus/BigLetters/SmallLetters/Taskg', 'bigLetter': 's', 'smallLetter': 's', 'ta

In [5]:
condition_names = list(conditions.keys()) # get the condition names as a list
condition_names_bids = [condition['BIDS_events'] for condition in conditions.values()] # get the condition names in bids format
condition_names

['Stimulus/BigLetters/SmallLetterh/Taskg',
 'Stimulus/BigLetters/SmallLetterh/Taskl',
 'Stimulus/BigLetters/SmallLetters/Taskg',
 'Stimulus/BigLetters/SmallLetters/Taskl',
 'Stimulus/BigLetterh/SmallLetterh/Taskg',
 'Stimulus/BigLetterh/SmallLetterh/Taskl',
 'Stimulus/BigLetterh/SmallLetters/Taskg',
 'Stimulus/BigLetterh/SmallLetters/Taskl']

get significant channels

In [6]:
sig_chans_per_subject = get_sig_chans_per_subject(subjects, epochs_root_file, task='GlobalLocal', LAB_root=None)

Loaded significant channels for subject D0057
Loaded significant channels for subject D0059
Loaded significant channels for subject D0063
Loaded significant channels for subject D0065
Loaded significant channels for subject D0069
Loaded significant channels for subject D0071
Loaded significant channels for subject D0077
Loaded significant channels for subject D0090
Loaded significant channels for subject D0094
Loaded significant channels for subject D0100
Loaded significant channels for subject D0102
Loaded significant channels for subject D0103


In [7]:
# Assuming you have a list of subjects
root_dir = rf"C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs"

In [8]:
# Define your ROIs dictionary and other parameters
rois_dict = {
    'lpfc': ["G_front_inf-Opercular", "G_front_inf-Orbital", "G_front_inf-Triangul", "G_front_middle", "G_front_sup", "Lat_Fis-ant-Horizont", "Lat_Fis-ant-Vertical", "S_circular_insula_ant", "S_circular_insula_sup", "S_front_inf", "S_front_middle", "S_front_sup"],
    'v1': ["G_oc-temp_med-Lingual", "S_calcarine", "G_cuneus"],
    'occ': ["G_cuneus", "G_and_S_occipital_inf", "G_occipital_middle", "G_occipital_sup", "G_oc-temp_lat-fusifor", "G_oc-temp_med-Lingual", "Pole_occipital", "S_calcarine", "S_oc_middle_and_Lunatus", "S_oc_sup_and_transversal", "S_occipital_ant"],
    'occ_filtered': [],
    'occ_best_filtered': []
}

rois = list(rois_dict.keys())
# Assuming you have subjects_electrodestoROIs_dict and sig_chans_per_subject dictionaries
electrodes_per_subject_roi, sig_electrodes_per_subject_roi, = make_sig_electrodes_per_subject_and_roi_dict(
    rois_dict, subjects_electrodestoROIs_dict, sig_chans_per_subject
)

For subject D0057, G_front_inf-Opercular, G_front_inf-Orbital, G_front_inf-Triangul, G_front_middle, G_front_sup, Lat_Fis-ant-Horizont, Lat_Fis-ant-Vertical, S_circular_insula_ant, S_circular_insula_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['RAI6', 'RAI12', 'RAI13', 'RAI15', 'RAI16', 'RPI14', 'RAMF10', 'RAMF11', 'RAMF12', 'RAIF11', 'RAIF12', 'RAIF13', 'RAIF14']
For subject D0059, G_front_inf-Opercular, G_front_inf-Orbital, G_front_inf-Triangul, G_front_middle, G_front_sup, Lat_Fis-ant-Horizont, Lat_Fis-ant-Vertical, S_circular_insula_ant, S_circular_insula_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['LMMF9', 'LMMF11', 'LMMF12', 'LPSF16']
For subject D0063, G_front_inf-Opercular, G_front_inf-Orbital, G_front_inf-Triangul, G_front_middle, G_front_sup, Lat_Fis-ant-Horizont, Lat_Fis-ant-Vertical, S_circular_insula_ant, S_circular_insula_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['LOF16', 'LASF10', 'LASF14', 'LASF15', 'LASF16', 'LM

let's manually make the occ filtered sig electrodes

In [9]:
sig_electrodes_per_subject_roi['occ_filtered'] = {
    'D0057': [],
    'D0059': [],
    'D0071': ['RO1', 'RO10'], #RO10 is iffy, big drop from fix onset
    'D0077': ['ROPM1', 'ROPM8'],
    'D0090': ['RTPO1', 'RTPI1'],
    'D0100': ['LOAI12', 'LOAI13', 'LOAI14', 'LOAI15', 'LOMI9', 'LOMI11', 'LOPI8', 'LOPI9', 'LOPM8', 'LOPM9', 'LPPI7', 'LPPI8', 'LPPI9'],
    'D0102': ['RTPI1'],
    'D0103': ['LTPI2', 'LTPI3', 'LTPI4']
}

and now make the best of the best

In [10]:
sig_electrodes_per_subject_roi['occ_best_filtered'] = {
    'D0057': [],
    'D0059': [],
    'D0071': [],
    'D0077': [],
    'D0090': [],
    'D0100': ['LOAI12', 'LOMI9', 'LOPI8', 'LOPI9', 'LOPM8', 'LOPM9'],
    'D0102': ['RTPI1'],
    'D0103': ['LTPI2', 'LTPI3', 'LTPI4']
}

In [11]:
# Specify the file path where you want to save the dictionary
save_path = 'sig_electrodes_per_subject_roi.json'

# Use json to save the dictionary
with open(save_path, 'w') as file:
    json.dump(sig_electrodes_per_subject_roi, file, indent=4)

print(f"Dictionary saved to {save_path}")

Dictionary saved to sig_electrodes_per_subject_roi.json


get electrode counts for each roi

In [12]:
total_electrodes_info = calculate_total_electrodes(sig_electrodes_per_subject_roi, electrodes_per_subject_roi)
for roi, counts in total_electrodes_info.items():
    print(f"Total number of {roi} electrodes across all subjects:", counts['total_electrodes'])
    print(f"Total number of significant {roi} electrodes across all subjects:", counts['total_significant_electrodes'])


Total number of lpfc electrodes across all subjects: 105
Total number of significant lpfc electrodes across all subjects: 46
Total number of v1 electrodes across all subjects: 17
Total number of significant v1 electrodes across all subjects: 12
Total number of occ electrodes across all subjects: 57
Total number of significant occ electrodes across all subjects: 41
Total number of occ_filtered electrodes across all subjects: 0
Total number of significant occ_filtered electrodes across all subjects: 23
Total number of occ_best_filtered electrodes across all subjects: 0
Total number of significant occ_best_filtered electrodes across all subjects: 10


check if any subjects have a weird sampling rate

In [13]:
# Assuming 'subjects_mne_objects' is your dictionary containing MNE objects for each subject
sampling_rate = 256
subject_rates = check_sampling_rates(subjects_mne_objects, expected_sampling_rate=sampling_rate)


Subject D0057 has the expected sampling rate: 256.0 Hz.
Subject D0059 has the expected sampling rate: 256.0 Hz.
Subject D0063 has the expected sampling rate: 256.0 Hz.
Subject D0065 has the expected sampling rate: 256.0 Hz.
Subject D0069 has the expected sampling rate: 256.0 Hz.
Subject D0071 has the expected sampling rate: 256.0 Hz.
Subject D0077 has the expected sampling rate: 256.0 Hz.
Subject D0090 has the expected sampling rate: 256.0 Hz.
Subject D0094 has the expected sampling rate: 256.0 Hz.
Subject D0100 has the expected sampling rate: 256.0 Hz.
Subject D0102 has the expected sampling rate: 256.0 Hz.
Subject D0103 has the expected sampling rate: 256.0 Hz.


create a dat dict like in the Temporal example from rsatoolbox 7/19  
#### Data Dictionary (dat) Structure
- **roi1**
  - **data**: 3D data array for this ROI
  - **channel_names**: List of significant channels for this ROI
  - **cond_names**: Dictionary where keys are condition names and values are integer indices
  - **cond_idx**: 1D array where each entry is an integer index corresponding to a cond_name
  - **times**: 1D array of times, corresponding to each sample
- **roi2**
  - **data**: 3D data array for this ROI
  - **channel_names**: List of significant channels for this ROI
  - **cond_names**: Dictionary where keys are condition names and values are integer indices
  - **cond_idx**: 1D array where each entry is an integer index corresponding to a cond_name
  - **times**: 1D array of times, corresponding to each sample
- **roiX**
  - **data**: 3D data array for this ROI
  - **channel_names**: List of significant channels for this ROI
  - **cond_names**: Dictionary where keys are condition names and values are integer indices
  - **cond_idx**: 1D array where each entry is an integer index corresponding to a cond_name
  - **times**: 1D array of times, corresponding to each sample


        




In [14]:
# Example call to the function (you need to replace the arguments with actual data)
dat = prepare_data_for_temporal_dataset(subjects_mne_objects, condition_names, rois, subjects, sig_electrodes_per_subject_roi)

subjects:  ['D0057', 'D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094', 'D0100', 'D0102', 'D0103']
num channels:  46
Processing D0057 for Stimulus/BigLetters/SmallLetterh/Taskg in lpfc
Number of trials for D0057 in Stimulus/BigLetters/SmallLetterh/Taskg: 59
sub:  D0057
channel indices:  [0, 1]
Processing D0057 for Stimulus/BigLetters/SmallLetterh/Taskl in lpfc
Number of trials for D0057 in Stimulus/BigLetters/SmallLetterh/Taskl: 54
sub:  D0057
channel indices:  [0, 1]
Processing D0057 for Stimulus/BigLetters/SmallLetters/Taskg in lpfc
Number of trials for D0057 in Stimulus/BigLetters/SmallLetters/Taskg: 53
sub:  D0057
channel indices:  [0, 1]
Processing D0057 for Stimulus/BigLetters/SmallLetters/Taskl in lpfc
Number of trials for D0057 in Stimulus/BigLetters/SmallLetters/Taskl: 34
sub:  D0057
channel indices:  [0, 1]
Processing D0057 for Stimulus/BigLetterh/SmallLetterh/Taskg in lpfc
Number of trials for D0057 in Stimulus/BigLetterh/SmallLetterh/Taskg: 51
sub:  D00

make roi_labeled_arrays, a dict where the keys are rois and the values are LabeledArrays. Index the same way as a nested dict. Use .labels to get labels from current level.


#### roi_labeled_arrays structure
- **roi1**: ROI name, string
  - **conditions**: condition name
    - **trials**: This is the maximal number of trials across subjects, filled with nans for subjects who don't have this many trials
      - **channels**: This is the number of channels in the roi, each channel is labeled as subject-channel name. Concatenated across subjects.
        - **samples**: 1 sample as a float. This is the time for this sample.

- **roi2**: ROI name, string
  - **conditions**: condition name
    - **trials**: This is the maximal number of trials across subjects, filled with nans for subjects who don't have this many trials
      - **channels**: This is the number of channels in the roi, each channel is labeled as subject-channel name. Concatenated across subjects.
        - **samples**: 1 sample as a float. This is the time for this sample.

- **roiX**: ROI name, string
  - **conditions**: condition name
    - **trials**: This is the maximal number of trials across subjects, filled with nans for subjects who don't have this many trials
      - **channels**: This is the number of channels in the roi, each channel is labeled as subject-channel name. Concatenated across subjects.
        - **samples**: 1 sample as a float. This is the time for this sample.


In [15]:
import numpy as np

def put_data_in_labeled_array_per_roi_subject(subjects_mne_objects, condition_names, rois, subjects, sig_electrodes_per_subject_roi):
    """
    Organize the MNE data into separate LabeledArray for each ROI and subject, with NaN padding for trial dimension.
    Concatenates subject data along the channels axis.
    
    Parameters:
    - subjects_mne_objects: Dictionary of MNE objects, structured as {subject: {condition: MNE epoch objects}}
    - condition_names: List of condition names.
    - rois: List of region of interest (ROI) names.
    - subjects: List of subjects.
    - sig_electrodes_per_subject_roi: Dictionary that maps ROIs to subjects and their corresponding electrodes.
    
    Returns:
    - roi_labeled_arrays: Dictionary of LabeledArrays for each ROI. 
                          Each LabeledArray concatenates the subject data along the channels dimension.
                          Dimensions of LabeledArray: [Conditions, Electrodes (concatenated), Trials, Timepoints]
    """
    roi_labeled_arrays = {}

    # Loop through each ROI
    for roi in rois:
        roi_labeled_array = None

        # Find the maximum number of trials for each condition across all subjects
        max_trials_per_condition = {condition: 0 for condition in condition_names}

        # First pass to find the max number of trials per condition
        for sub in subjects:
            sig_electrodes = sig_electrodes_per_subject_roi.get(roi, {}).get(sub, [])
            if not sig_electrodes:
                continue
            for condition_name in condition_names:
                epochs = subjects_mne_objects[sub][condition_name]['HG_ev1_rescaled'].copy().pick(sig_electrodes)
                epochs_data = epochs.get_data(copy=True)  # Shape: (trials, channels, timepoints)
                max_trials_per_condition[condition_name] = max(max_trials_per_condition[condition_name], epochs_data.shape[0])

        # Second pass to gather data for each subject and condition
        for sub in subjects:
            sig_electrodes = sig_electrodes_per_subject_roi.get(roi, {}).get(sub, [])
            if not sig_electrodes:
                continue

            subject_nested_dict = {}

            # Get channel names for this subject's ROI
            sub_channel_names = [f"{sub}-{sig_electrode}" for sig_electrode in sig_electrodes]

            # Loop through each condition
            for condition_name in condition_names:
                # Extract the epoch data for the current condition and subject, pick only the electrodes of interest
                epochs = subjects_mne_objects[sub][condition_name]['HG_ev1_rescaled'].copy().pick(sig_electrodes)
                epochs_data = epochs.get_data(copy=True)  # Shape: (trials, channels, timepoints)
                # Get the number of trials and the target number of trials
                n_trials, n_channels, n_timepoints = epochs_data.shape
                max_trials = max_trials_per_condition[condition_name]

                # Pad with NaNs if necessary
                if n_trials < max_trials:
                    # Create a padded array with NaNs
                    padded_data = np.full((max_trials, n_channels, n_timepoints), np.nan)
                    padded_data[:n_trials, :, :] = epochs_data  # Copy existing data into the padded array
                else:
                    padded_data = epochs_data

                subject_nested_dict[condition_name] = padded_data
            times = epochs.times # same times across conditions, so just grab times of last condition
            str_times = [str(time) for time in times]
            np_array_str_times = np.array(str_times) # make this a numpy array of strings so it plays nice with aaron's code

            # Create a LabeledArray for the subject
            subject_labeled_array = LabeledArray.from_dict(subject_nested_dict)
            subject_labeled_array.labels[2] = sub_channel_names  # Add channel labels for this subject
            subject_labeled_array.labels[3] = np_array_str_times # Add time labels for this subject. This breaks things. But why????? 9/10. 
            
            if roi_labeled_array is None:
                roi_labeled_array = subject_labeled_array
            else:
                roi_labeled_array = roi_labeled_array.concatenate(subject_labeled_array, axis=2) # concatenate along channels axis

        # Add the concatenated LabeledArray to the ROI dictionary
        roi_labeled_arrays[roi] = roi_labeled_array

    return roi_labeled_arrays


In [16]:
# Example call to the function
roi_labeled_arrays = put_data_in_labeled_array_per_roi_subject(subjects_mne_objects, condition_names, rois, subjects, sig_electrodes_per_subject_roi)

set up time bins, time indexing, array reshaping, and decoding functions 9/12

In [18]:
def create_time_bins(time_points, bin_size=0.1):
    """
    Create time bins from the given time points array.

    Parameters:
    - time_points: 1D numpy array of time points.
    - bin_size: The size of each time bin in seconds (default is 0.1s, which is 100 ms).

    Returns:
    - bins: A list of numpy arrays, where each array contains the time points within that bin.
    """
    bins = []
    start_time = np.min(time_points)
    end_time = np.max(time_points)
    current_time = start_time
    
    while current_time < end_time:
        next_time = current_time + bin_size
        if next_time > end_time:
            next_time = end_time
        bins.append(time_points[(time_points >= current_time) & (time_points < next_time)])
        current_time = next_time

    # Ensure that the last bin includes the endpoint
    if time_points[-1] not in bins[-1]:
        bins[-1] = np.append(bins[-1], time_points[-1])

    return bins

def get_bin_centers(time_bins):
    """
    Calculate the center of each time bin.

    Parameters:
    - time_bins: List of numpy arrays, where each array represents a time bin.

    Returns:
    - bin_centers: A list of bin center values (floats).
    """
    bin_centers = [np.mean(bin) for bin in time_bins]
    return bin_centers


In [30]:
def index_timepoints_by_bins(labeled_array, time_bins):
    """
    Index time points from a LabeledArray using given time bins.

    Parameters:
    - labeled_array: The LabeledArray from which to extract timepoints (e.g., roi_labeled_arrays['lpfc']).
    - time_bins: List of numpy arrays, where each array represents a time bin.

    Returns:
    - bin_indices: A list of lists, where each list contains the indices of the time points that fall into the corresponding bin.
    """
    # Get the time points from the LabeledArray
    time_points = labeled_array.labels[3].astype(float)  # Convert the time labels to floats

    bin_indices = []
    
    # Loop over the time bins and find the corresponding indices
    for time_bin in time_bins:
        indices_in_bin = np.where(np.isin(time_points, time_bin))[0]  # Get indices of time points in this bin
        bin_indices.append(indices_in_bin)
    
    return bin_indices

In [31]:
def reshape_labeled_array_for_decoding(labeled_array):
    """
    Reshape a single LabeledArray, removing trials that have NaN values and reshaping to be trials by channels*timepoints.

    Parameters:
    - labeled_array: A LabeledArray where the 0th axis labels are conditions, 1st axis labels are trials, 
                     2nd axis labels are channels, and 3rd axis labels are timepoints.

    Returns:
    - reshaped_data: Dictionary where keys are condition names and values are reshaped arrays (trials x channels*timepoints).
    """
    reshaped_labeled_array = {}
    
    # Extract the condition labels (which are in labeled_array.labels[0])
    condition_names = labeled_array.labels[0]

    # Loop over each condition
    for condition_name in condition_names:
        # Extract the data for the current condition
        condition_data = labeled_array[condition_name]
        
        # Find the indices of trials that do not contain NaNs
        valid_trial_indices = ~np.isnan(condition_data).any(axis=(1, 2))

        # Use the take method to select only the valid trials
        condition_data_clean = condition_data.take(indices=np.where(valid_trial_indices)[0], axis=0)
        
        # Combine the channels and timepoints axes (axis 1 and axis 2) into a single axis
        channel_time_combined_array = condition_data_clean.reshape(condition_data_clean.shape[0], -1)
        
        # Store the processed data for this condition
        reshaped_labeled_array[condition_name] = channel_time_combined_array

    return reshaped_labeled_array

def process_all_rois_for_decoding(roi_labeled_arrays):
    """
    Loop through all ROIs and apply reshaping function to each LabeledArray.

    Parameters:
    - roi_labeled_arrays: Dictionary of LabeledArrays for each ROI.

    Returns:
    - all_processed_data: Dictionary where keys are ROIs and values are dictionaries of reshaped data for each condition.
    """
    reshaped_roi_labeled_arrays_for_decoding = {}

    # Loop through each ROI in the dictionary
    for roi, labeled_array in roi_labeled_arrays.items():
        # Apply the reshaping function to the current labeled array
        reshaped_labeled_array = reshape_labeled_array_for_decoding(labeled_array)

        # Store the reshaped data for this ROI
        reshaped_roi_labeled_arrays_for_decoding[roi] = reshaped_labeled_array

    return reshaped_roi_labeled_arrays_for_decoding


In [32]:
def concatenate_conditions_by_string(roi_labeled_arrays, roi, strings_to_find):
    """
    Concatenate trials across condition names that contain specific strings.
    Assign labels based on the groupings of the conditions.

    Parameters:
    - roi_labeled_arrays: Dictionary of LabeledArrays for each ROI.
    - roi: The specific ROI to process.
    - strings_to_find: List of strings to search for in condition names.

    Returns:
    - concatenated_data: The concatenated trials by channels*timepoints across the matching conditions.
    - labels: A numpy array of labels (0, 1, 2, ...) corresponding to each group of conditions.
    - cats: Dictionary of {condition_name: index} for decoding.
    """
    concatenated_data = []
    labels = []
    cats = {}

    # Track current label index
    current_label = 0

    for string in strings_to_find:
        # Find condition names containing the string
        matching_conditions = [cond for cond in roi_labeled_arrays[roi].keys() if string in cond]

        if not matching_conditions:
            continue

        # Concatenate data for all matching conditions
        data_to_concatenate = []
        for cond in matching_conditions:
            # Extract data for the current condition
            data = roi_labeled_arrays[roi][cond]  # Shape: (trials, channels, timepoints)
            data_to_concatenate.append(data)
            
            # Update labels
            labels.extend([current_label] * data.shape[0])
        
        # Check if we have data to concatenate for this condition
        if data_to_concatenate:
            concatenated_data.append(np.concatenate(data_to_concatenate, axis=0))

        # Assign current label to the condition group
        cats[string] = current_label
        current_label += 1

    # Ensure there is data to concatenate
    if not concatenated_data:
        raise ValueError(f"No matching conditions found for ROI: {roi} and strings: {strings_to_find}")

    # Concatenate all condition data along the trials axis
    concatenated_data = np.concatenate(concatenated_data, axis=0)
    
    return concatenated_data, np.array(labels), cats

set up reshaped arrays for each time bin 9/12

In [33]:
# # Assuming temporal_datasets is a dictionary
# first_sub = next(iter(subjects_mne_objects.keys()))
# first_cond = next(iter(subjects_mne_objects[first_sub].keys()))

# time_points = subjects_mne_objects[first_sub][first_cond]['HG_ev1_rescaled'].times

# # Create time bins
# time_bins = create_time_bins(time_points, bin_size=0.1)

# # get bin centers for plotting uses
# bin_centers = get_bin_centers(time_bins)

# # Create time-binned LabeledArrays for each ROI
# first_roi = next(iter(roi_labeled_arrays.keys()))
# time_binned_roi_labeled_arrays = {}
# bin_indices = index_timepoints_by_bins(roi_labeled_arrays[first_roi], time_bins)
# bin_centers = get_bin_centers(time_bins)  # Get the bin centers for plotting


# # for each time bin, make a reshaped roi labeled arrays object with just the time indices from this bin
# reshaped_roi_labeled_arrays_all_bins = {}

# # go through each time bin and make a reshaped array for it
# for i, indices in enumerate(bin_indices):
#     roi_labeled_arrays_this_bin = {}
#     bin_center = bin_centers[i] # this is the bin center for plotting purposes

#     for roi in rois:
#         # index the roi labeled array and grab just these time indices
#         roi_labeled_arrays_this_bin[roi] = roi_labeled_arrays[roi][:, :, :, indices]

#     # drop trial nans and concatenate channel-time for the time indexed arrays
#     reshaped_roi_labeled_arrays_for_decoding_this_bin = process_all_rois_for_decoding(roi_labeled_arrays_this_bin)

#     # store this reshaped array with the bin center as a key
#     reshaped_roi_labeled_arrays_all_bins[bin_center] = reshaped_roi_labeled_arrays_for_decoding_this_bin

fancy way of setting up reshaped arrays for each time bin using a class instead!

In [46]:
class TimeBinnedROILabeledArray:
    def __init__(self, bin_center, indices, data):
        """
        Initialize a TimeBinnedROILabeledArray object.

        Parameters:
        - bin_center: The center of the time bin.
        - indices: The indices of the time points that belong to this bin.
        - data: The reshaped ROI labeled arrays for decoding.
        """
        self.bin_center = bin_center
        self.indices = indices
        self.data = data

class TimeBinnedArraysDataManager:
    def __init__(self, roi_labeled_arrays, time_bins, rois):
        """
        Initialize the class to manage all time-binned reshaped ROI labeled arrays.

        Parameters:
        - roi_labeled_arrays: Dictionary of LabeledArrays for each ROI.
        - time_bins: The time bins generated for the data.
        - rois: The regions of interest to process.
        """
        self.roi_labeled_arrays = roi_labeled_arrays
        self.time_bins = time_bins
        self.rois = rois
        self.bin_centers = get_bin_centers(time_bins)
        self.bin_indices = None
        self.binned_data = []  # List of TimeBinnedROILabeledArray objects

    def index_time_bins(self):
        """Index the time points for each bin."""
        first_roi = next(iter(self.roi_labeled_arrays.keys()))
        self.bin_indices = index_timepoints_by_bins(self.roi_labeled_arrays[first_roi], self.time_bins)

    def process_bins(self):
        """Process each time bin and reshape the ROI labeled arrays for decoding."""
        self.index_time_bins()  # Ensure bin indexing is done before processing bins

        for i, indices in enumerate(self.bin_indices):
            bin_center = self.bin_centers[i]
            roi_labeled_arrays_this_bin = {}

            for roi in self.rois:
                # Index the ROI labeled array and grab just these time indices
                roi_labeled_arrays_this_bin[roi] = self.roi_labeled_arrays[roi][:, :, :, indices]

            # Drop trial NaNs and concatenate channel-time for the time-indexed arrays
            reshaped_roi_labeled_arrays_for_decoding_this_bin = process_all_rois_for_decoding(roi_labeled_arrays_this_bin)

            # Create a TimeBinnedROILabeledArray object and store it
            self.binned_data.append(TimeBinnedROILabeledArray(bin_center, indices, reshaped_roi_labeled_arrays_for_decoding_this_bin))

    def get_binned_data(self):
        """Return the list of TimeBinnedROILabeledArray objects."""
        return self.binned_data

# make the wrapper class first with all the attributes, then get the list of individual time binned arrays. Name better?
TimeBinnedArraysDataManager = TimeBinnedArraysDataManager(roi_labeled_arrays, time_bins, rois)
TimeBinnedArraysDataManager.process_bins()
time_binned_arrays = TimeBinnedArraysDataManager.get_binned_data()

# Print the first bin's center for example
time_binned_arrays[0].data


{'lpfc': {'Stimulus/BigLetters/SmallLetterh/Taskg': array([[-0.22146894, -0.36678028, -0.45204559, ...,  0.4783572 ,
           0.52571112,  0.56830347],
         [-0.23705637, -0.05304538,  0.15468368, ..., -0.72919285,
          -0.62866354, -0.53704494],
         [-0.60417521, -0.35751507, -0.10712799, ...,  0.40507853,
           0.35642576,  0.28808269],
         ...,
         [-1.07376695, -1.07829988, -1.08036458, ..., -0.27965099,
          -0.16478485, -0.05605476],
         [ 0.64907086,  0.46285841,  0.34506354, ...,  0.89371949,
           0.64035177,  0.3651692 ],
         [ 0.49512956,  0.56156552,  0.61483103, ..., -0.95983958,
          -0.91317755, -0.86310685]])
  labels(['0', '1', '2', '3', '4', '6', '7', '9', '10', '12', '13', '14', '15', '16', '17', '18', '20', '21', '23', '24', '25']
         ['D0057-RAI6', 'D0057-RPI14', 'D0059-LMMF9', 'D0059-LMMF11', 'D0059-LMMF12', 'D0059-LPSF16', 'D0063-LMSF5', 'D0063-LPSF12', 'D0063-RAI4', 'D0063-RAI6', 'D0063-RMMF13', 'D0063

set up decoding output directory and conditions to compare 9/12

In [47]:
# Determine LAB_root based on the operating system
HOME = os.path.expanduser("~")
LAB_root = os.path.join(HOME, "Box", "CoganLab") if os.name == 'nt' else os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")
save_dir = os.path.join(LAB_root, 'BIDS-1.1_GlobalLocal', 'BIDS', 'derivatives', 'decoding', 'figs')

strings_to_find = ['SmallLetters', 'SmallLetterh']  # Strings to search for in condition names

do decoding on the reshaped time indexed arrays 9/12

In [ ]:
for time_binned_array in time_binned_arrays:
    for roi in rois:
        print(roi)
        
        # Concatenate the trials and get labels
        concatenated_data, labels, cats = concatenate_conditions_by_string(time_binned_array.data, roi, strings_to_find)
        print(concatenated_data.shape)
        for string in strings_to_find:
            num_trials = len([labels for label in labels if label==cats[string]])
            print(f"{string} has {num_trials} trials")

        # Create a Decoder and run cross-validation
        decoder = Decoder(cats, 0.80, oversample=True, n_splits=5, n_repeats=5)

        # Use the concatenated data for the decoder
        cm = decoder.cv_cm(concatenated_data, labels, normalize='true') # uh how to get the individual decoding scores out of this so i can plot them?? 9/12
        cm = np.mean(cm, axis=0)

        bin_center = time_binned_array.bin_center # the centered x-value on which to plot these decoding scores. Actually just get this from the data manager class as the x, and the y's i can just append to a list cuz the length will be the same.

do decoding across all timepoints

In [101]:
reshaped_roi_labeled_arrays_for_decoding = process_all_rois_for_decoding(roi_labeled_arrays)

for roi in rois:
    print(roi)
    
    # Concatenate the trials and get labels
    concatenated_data, labels, cats = concatenate_conditions_by_string(reshaped_roi_labeled_arrays_for_decoding, roi, strings_to_find)
    print(concatenated_data.shape)
    for string in strings_to_find:
        num_trials = len([labels for label in labels if label==cats[string]])
        print(f"{string} has {num_trials} trials")

    # Create a Decoder and run cross-validation
    decoder = Decoder(cats, 0.80, oversample=True, n_splits=5, n_repeats=5)

    # Use the concatenated data for the decoder
    cm = decoder.cv_cm(concatenated_data, labels, normalize='true')
    cm = np.mean(cm, axis=0)

    # Plot the Confusion Matrix
    fig, ax = plt.subplots()
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=cats.keys())
    disp.plot(ax=ax)

    # Save the figure with the strings_to_find reflected in the filename
    file_name = f'{roi}_{"_vs_".join(strings_to_find)}_time_averaged_confusion_matrix.png'
    plt.savefig(os.path.join(save_dir, file_name))
    plt.close()

lpfc
(210, 29486)
SmallLetters has 103 trials
SmallLetterh has 107 trials
v1
(258, 7692)
SmallLetters has 142 trials
SmallLetterh has 116 trials
occ
(193, 26281)
SmallLetters has 98 trials
SmallLetterh has 95 trials
occ_filtered
(247, 14743)
SmallLetters has 122 trials
SmallLetterh has 125 trials
occ_best_filtered
(277, 6410)
SmallLetters has 140 trials
SmallLetterh has 137 trials


now try to do this for different time windows. This is not going to work, think through how to go from channel-sample labels to time bins more. 9/8

why are timepoint labels broken ugh...

In [ ]:
# # this is wrong because we start from -1, remember? This assumes start from 0. Let's use the time bins instead somehow. Ugh why does epochs.times break the labels??? 9/10
# def get_sample_indices_for_time_window(time_start, time_end, sfreq, n_samples):
#     """
#     Get the sample indices corresponding to a time window.

#     Parameters:
#     - time_start: The start time of the window in seconds.
#     - time_end: The end time of the window in seconds.
#     - sfreq: The sampling frequency (samples per second).
#     - n_samples: The total number of samples in the data.

#     Returns:
#     - start_idx, end_idx: The sample indices corresponding to the time window.
#     """
#     start_idx = int(np.floor(time_start * sfreq))
#     end_idx = int(np.ceil(time_end * sfreq))
#     end_idx = min(end_idx, n_samples)  # Ensure we don't go beyond the available samples
#     return start_idx, end_idx

# def run_decoding_on_time_windows(reshaped_data, labels, sfreq, window_size=0.5):
#     """
#     Run decoding for each time window and calculate accuracy over time.

#     Parameters:
#     - reshaped_data: Dictionary with condition names as keys and reshaped arrays as values.
#     - labels: Array of condition labels.
#     - sfreq: Sampling frequency in Hz.
#     - window_size: The size of the time window in seconds (default 500ms).

#     Returns:
#     - decoding_scores: List of decoding accuracies for each time window.
#     - time_points: List of time midpoints corresponding to each window.
#     """
#     n_samples = reshaped_data[list(reshaped_data.keys())[0]].shape[1]  # Assuming all conditions have the same shape
#     total_time = n_samples / sfreq
#     decoding_scores = []
#     time_points = []

#     # Loop over time windows
#     time_start = 0
#     while time_start < total_time:
#         time_end = time_start + window_size
#         time_midpoint = (time_start + time_end) / 2
#         time_points.append(time_midpoint)

#         # Get the sample indices for the current time window
#         start_idx, end_idx = get_sample_indices_for_time_window(time_start, time_end, sfreq, n_samples)

#         # Reshape data for the current time window
#         data_for_window = []
#         for condition_name in reshaped_data:
#             # Extract channels-timepoints for the specific time window
#             data_window = reshaped_data[condition_name][:, start_idx:end_idx]
#             data_for_window.append(data_window)

#         concatenated_data = np.concatenate(data_for_window, axis=0)

#         # Run decoding (simplified for this example)
#         decoder = Decoder(categories, 0.80, oversample=True, n_splits=5, n_repeats=5)
#         cm = decoder.cv_cm(concatenated_data, labels, normalize='true')
#         cm = np.mean(cm, axis=0)

#         # Calculate the accuracy
#         accuracy = np.trace(cm) / np.sum(cm)
#         decoding_scores.append(accuracy)

#         # Move to the next time window
#         time_start += window_size

#     return decoding_scores, time_points

# def plot_decoding_scores(time_points, decoding_scores):
#     """
#     Plot decoding accuracy over time.

#     Parameters:
#     - time_points: List of time midpoints.
#     - decoding_scores: List of decoding accuracies.
#     """
#     plt.figure(figsize=(10, 6))
#     plt.plot(time_points, decoding_scores, marker='o')
#     plt.xlabel('Time (s)')
#     plt.ylabel('Decoding Accuracy')
#     plt.title('Decoding Accuracy Over Time')
#     plt.grid(True)
#     plt.show()

# # Example usage:
# sfreq = 2048  # should be 2048 for all subjects
# window_size = 0.5  # Window size in seconds

# # Assuming reshaped_roi_labeled_arrays_for_decoding and labels are prepared
# decoding_scores, time_points = run_decoding_on_time_windows(reshaped_roi_labeled_arrays_for_decoding[roi], labels, sfreq, window_size)

# # Plot the decoding accuracy over time
# plot_decoding_scores(time_points, decoding_scores)


define colors for plotting (not used yet as of 8/21)

In [ ]:
# Define colors for the model names
colors = {
    'congruency': 'red',
    'switchType': 'blue',
    'congruencyProportion': 'pink',
    'switchProportion': 'skyblue',
    'congruency_congruencyProportion': 'hotpink',
    'congruency_congruency_proportion': 'hotpink',
    'switchType_switchProportion': 'gray',
    'switch_type_switch_proportion': 'gray',
    'bigLetter': 'green',
    'big_letter': 'green',
    'smallLetter': 'orange',
    'small_letter': 'orange',
    'task': 'gray',
    'c75.0': 'pink',
    'i75.0': 'pink',
    'c25.0': 'gold',
    'i25.0': 'gold',
    'r25.0': 'lightblue',
    's25.0': 'lightblue',
    'r75.0': 'purple',
    's75.0': 'purple'
}

# Define linestyles for the model names
linestyles = {
    'big letter S': '-',
    'BigLetters': '-',

    'big letter H': '--',
    'BigLetterh': '--',

    'small letter S': '-',
    'SmallLetters': '-',

    'small letter H': '--',
    'SmallLetterh': '--',

    'task G': '-',
    'Taskg': '-',

    'task L': '--',
    'Taskl': '--',

    'congruent': '-',
    'c': '-',

    'incongruent': '--',
    'i': '--',

    'repeat': '-',
    'r': '-',

    'switch': '--',
    's': '--',

    'c25.0': '-',
    'c75.0': '-',
    'i25.0': '--',
    'i75.0': '--'
}